In [ ]:
import pandas as pd
import requests
from shapely.geometry import Polygon, MultiPolygon
from pyproj import Geod, Proj

url_stations = 'https://api.energyandcleanair.org/stations?country=GB,US,TR,PH,IN,TH&format=geojson'
response = requests.get(url_stations)
data = response.json()
df = pd.json_normalize(data['features'])

In [2]:
df.columns

Index(['type', 'geometry.type', 'geometry.coordinates', 'properties.gpw',
       'properties.level', 'properties.gadm1_id', 'properties.last_updated',
       'properties.timezone', 'properties.first_updated',
       'properties.gadm2_id', 'properties.last_scraped_data',
       'properties.type', 'properties.attribution', 'properties.country_id',
       'properties.name', 'properties.infos.sources',
       'properties.infos.isMobile', 'properties.infos.isAnalysis',
       'properties.infos.sensor', 'properties.infos.entity',
       'properties.source', 'properties.names', 'properties.show_in_dashboard',
       'properties.city_id', 'properties.id', 'properties.pollutants',
       'properties.city_name', 'properties.infos.city_code',
       'properties.infos.TownId', 'properties.infos.DataPeriods',
       'properties.infos.StationSubType', 'properties.infos.CityId',
       'properties.infos.SectorCode', 'properties.infos.Station_Title',
       'properties.infos.StationGroup', 'properties

In [3]:
df['properties.pollutants']

0       [pm10, pm25]
1             [pm10]
2               [o3]
3             [pm25]
4         [o3, pm25]
            ...     
6099       [no2, o3]
6100    [pm10, pm25]
6101           [no2]
6102           [no2]
6103       [no2, o3]
Name: properties.pollutants, Length: 6104, dtype: object

In [4]:
# Filter PM10 monitoring stations
stations_with_pm10 = df[df['properties.pollutants'].apply(lambda x: 'pm10' in x if isinstance(x, list) else False)]

In [5]:
target_countries = df['properties.country_id'].unique()
target_countries

array(['US', 'TR', 'GB', 'TH', 'PH', 'IN'], dtype=object)

In [6]:
url_country_boundaries = 'https://datahub.io/core/geo-countries/r/countries.geojson'
response_country = requests.get(url_country_boundaries)
data_country = response_country.json()
country_df = pd.json_normalize(data_country['features'])
country_df.head()

,type,properties.ADMIN,properties.ISO_A3,properties.ISO_A2,geometry.type,geometry.coordinates
0,Feature,Aruba,ABW,AW,MultiPolygon,"[[[[-69.99693762899992, 12.577582098000036], [..."
1,Feature,Afghanistan,AFG,AF,MultiPolygon,"[[[[71.04980228700009, 38.40866445000009], [71..."
2,Feature,Angola,AGO,AO,MultiPolygon,"[[[[11.73751945100014, -16.692577982999836], [..."
3,Feature,Anguilla,AIA,AI,MultiPolygon,"[[[[-63.037668423999946, 18.21295807500003], [..."
4,Feature,Albania,ALB,AL,MultiPolygon,"[[[[19.747765747000074, 42.57890085900007], [1..."


In [7]:
filtered_countries_df = country_df[country_df['properties.ISO_A2'].isin(target_countries)]
filtered_countries_df

,type,properties.ADMIN,properties.ISO_A3,properties.ISO_A2,geometry.type,geometry.coordinates
81,Feature,United Kingdom,GBR,GB,MultiPolygon,"[[[[-6.287505662999905, 49.91400788000006], [-..."
104,Feature,India,IND,IN,MultiPolygon,"[[[[93.85531660200016, 7.214178778000104], [93..."
179,Feature,Philippines,PHL,PH,MultiPolygon,"[[[[119.84978274800014, 4.796861070000134], [1..."
223,Feature,Thailand,THA,TH,MultiPolygon,"[[[[99.247813347, 6.57485586100016], [99.26026..."
230,Feature,Turkey,TUR,TR,MultiPolygon,"[[[[26.04004967500012, 39.84503815300015], [26..."
238,Feature,United States of America,USA,US,MultiPolygon,"[[[[-155.6065189699999, 20.137955566000144], [..."


In [8]:
geod = Geod('+a=6378137 +f=0.0033528106647475126')

# Apply display setting for numbers in pandas
pd.options.display.float_format = '{:,.2f}'.format

# Function to calculate the area of ​​a country
def area_perim(country_name):
    # Extract country geometry by its ISO code
    country_row = filtered_countries_df[filtered_countries_df["properties.ISO_A2"] == country_name]
    coords = country_row['geometry.coordinates'].iloc[0]
    
    total_area = 0  # Total area
    if isinstance(coords[0][0], list):  # MultiPolygon
        for polygon in coords:
            lon, lat = zip(*polygon[0])  # Coordinates of the outer contour of the polygon
            poly_area, _ = geod.polygon_area_perimeter(lon, lat)
            total_area += abs(poly_area)  # We sum up the absolute values ​​of the areas
    else:
        lon, lat = zip(*coords[0])  # Coordinates of a single polygon
        total_area, _ = geod.polygon_area_perimeter(lon, lat)
    
    return total_area / 10**6  # Area in sq. km.


# Create a dictionary with areas for each country
area_dict = {country: area_perim(country) for country in target_countries}

In [9]:
stations_count = stations_with_pm10['properties.country_id'].value_counts().reset_index()
stations_count.columns = ['properties.ISO_A2', 'Number of PM10 Stations']
stations_count

,properties.ISO_A2,Number of PM10 Stations
0,US,1035
1,IN,536
2,TR,332
3,GB,165
4,TH,162
5,PH,22


In [11]:
# Create a DataFrame with the number of PM10 stations and area for each country
result_df = filtered_countries_df[filtered_countries_df['properties.ISO_A2'].isin(target_countries)][['properties.ISO_A2']]

# Add a column with the area for each country
result_df['Area (sq km)'] = result_df['properties.ISO_A2'].map(area_dict)
result_df

,properties.ISO_A2,Area (sq km)
81,GB,"243,783.21"
104,IN,"3,151,481.06"
179,PH,"293,237.49"
223,TH,"514,453.69"
230,TR,"780,079.65"
238,US,"9,464,457.28"


In [12]:
# Calculate the number of PM10 stations for each country
stations_count = stations_with_pm10['properties.country_id'].value_counts().reset_index()
stations_count.columns = ['properties.ISO_A2', 'Number of PM10 Stations']

# Add a column with the number of PM10 stations
result_df = result_df.merge(stations_count, on='properties.ISO_A2', how='left')
result_df

,properties.ISO_A2,Area (sq km),Number of PM10 Stations
0,GB,"243,783.21",165
1,IN,"3,151,481.06",536
2,PH,"293,237.49",22
3,TH,"514,453.69",162
4,TR,"780,079.65",332
5,US,"9,464,457.28",1035


In [14]:
# Calculating the density of PM10 stations per 1,000 sq. km
result_df['Density of PM10 Stations per 1,000 sq. km'] = result_df['Number of PM10 Stations'] / result_df['Area (sq km)'] * 1000

result_df

,properties.ISO_A2,Area (sq km),Number of PM10 Stations,"Density of PM10 Stations per 1,000 sq. km"
0,GB,"243,783.21",165,0.68
1,IN,"3,151,481.06",536,0.17
2,PH,"293,237.49",22,0.08
3,TH,"514,453.69",162,0.31
4,TR,"780,079.65",332,0.43
5,US,"9,464,457.28",1035,0.11


In [15]:
# Add column properties.ADMIN in result_df_sorted
result_df = result_df.merge(
    filtered_countries_df[['properties.ISO_A2', 'properties.ADMIN']],
    left_on='properties.ISO_A2',
    right_on='properties.ISO_A2',
    how='left'
)

# Rename the column
result_df.rename(columns={'properties.ADMIN': 'Country Name'}, inplace=True)

result_df = result_df[['Country Name', 'Number of PM10 Stations', 'Area (sq km)', 'Density of PM10 Stations per 1,000 sq. km']]
result_df

,Country Name,Number of PM10 Stations,Area (sq km),"Density of PM10 Stations per 1,000 sq. km"
0,United Kingdom,165,"243,783.21",0.68
1,India,536,"3,151,481.06",0.17
2,Philippines,22,"293,237.49",0.08
3,Thailand,162,"514,453.69",0.31
4,Turkey,332,"780,079.65",0.43
5,United States of America,1035,"9,464,457.28",0.11


In [18]:
# Remove old indexes and add a new column
result_df.reset_index(drop=True, inplace=True)
result_df.index += 1  
result_df.index.name = '№'  

result_df

,Country Name,Number of PM10 Stations,Area (sq km),"Density of PM10 Stations per 1,000 sq. km"
№,,,,
1,United Kingdom,165,"243,783.21",0.68
2,India,536,"3,151,481.06",0.17
3,Philippines,22,"293,237.49",0.08
4,Thailand,162,"514,453.69",0.31
5,Turkey,332,"780,079.65",0.43
6,United States of America,1035,"9,464,457.28",0.11
